In [1]:
from hdimvis.metrics.stress.stress import vectorised_stress,unvectorised_stress
from hdimvis.data_fetchers.DataFetcher import DataFetcher
from hdimvis.metrics.distance_measures.euclidian_and_manhattan import euclidean,manhattan
import numpy as np
import tracemalloc
from time import perf_counter
import definitions
import os
import matplotlib.pylab as plt

In [2]:
large_dataset = DataFetcher.fetch_data('mnist', size='max')


####################
Fetching the "mnist" dataset


C:\Users\Owner\.virtualenvs\2095736w-0SnFieZ0\lib\site-packages\sklearn\datasets\_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


####################
Dataset loaded
Dataset shape: (70000, 784)
####################


In [3]:
sizes = np.arange(10,2020, 20)
print(sizes)
# sizes = [3000]
num_repeats = 3

# 3 for memory use base (index 0), memory use peak (1) and time (index 2),
vectorised_data = np.zeros((len(sizes),3,num_repeats))
un_vectorised_data = np.zeros((len(sizes),3,num_repeats))

collected_data = [vectorised_data, un_vectorised_data]
functions = [vectorised_stress, unvectorised_stress]

[  10   30   50   70   90  110  130  150  170  190  210  230  250  270
  290  310  330  350  370  390  410  430  450  470  490  510  530  550
  570  590  610  630  650  670  690  710  730  750  770  790  810  830
  850  870  890  910  930  950  970  990 1010 1030 1050 1070 1090 1110
 1130 1150 1170 1190 1210 1230 1250 1270 1290 1310 1330 1350 1370 1390
 1410 1430 1450 1470 1490 1510 1530 1550 1570 1590 1610 1630 1650 1670
 1690 1710 1730 1750 1770 1790 1810 1830 1850 1870 1890 1910 1930 1950
 1970 1990 2010]


In [4]:
for i, size in enumerate(sizes):
    for j in range(2):
        for k in range(num_repeats):
            print(size)
            sample_indices = np.random.randint(0, 69999, size)
            sample = large_dataset.data[sample_indices]
            ld_positions = 20*np.random.rand(sample.shape[0],2)
            stress_fn = functions[j]

            tracemalloc.start()
            stress1 = stress_fn(sample, ld_positions, euclidean)
            collected_data[j][i,0,k] = tracemalloc.get_traced_memory()[0]
            collected_data[j][i,1,k] = tracemalloc.get_traced_memory()[1]
            tracemalloc.stop()

            start = perf_counter()
            stress2 = stress_fn(sample, ld_positions, euclidean)
            collected_data[j][i,2,k] = perf_counter() - start



output_dir = os.path.realpath(os.path.join(definitions.PROJECT_ROOT, "experiments/stress_vectorisation/out/stress_vec_1.npy"))

with open(output_dir, 'wb') as f:
    np.save(f, vectorised_data)
    np.save(f, un_vectorised_data)

10
10
10
10
10
10
30
30
30
30
30
30
50
50
50
50
50
50
70
70
70
70
70
70
90
90
90
90
90
90
110
110
110
110
110
110
130
130
130
130
130
130
150
150
150
150
150
150
170
170
170
170
170
170
190
190
190
190
190
190
210
210
210
210
210
210
230
230
230
230
230
230
250
250
250
250
250
250
270
270
270
270
270
270
290
290
290
290
290
290
310
310
310
310
310
310
330
330
330
330
330
330
350
350
350
350
350
350
370
370
370
370
370
370
390
390
390
390
390
390
410
410
410
410
410
410
430
430
430
430
430
430
450
450
450
450
450
450
470
470
470
470
470
470
490
490
490
490
490
490
510
510
510
510
510
510
530
530
530
530
530
530
550
550
550
550
550
550
570
570
570
570
570
570
590
590
590
590
590
590
610
610
610
610
610
610
630
630
630
630
630
630
650
650
650
650
650
650
670
670
670
670
670
670
690
690
690
690
690
690
710
710
710
710
710
710
730
730
730
730
730
730
750
750
750
750
750
750
770
770
770
770
770
770
790
790
790
790
790
790
810
810
810
810
810
810
830
830
830
830
830
830
850
850
850
850
850
85

In [5]:
output_dir = os.path.realpath(os.path.join(definitions.PROJECT_ROOT, "experiments/stress_vectorisation/out/stress_vec_1.npy"))

with open(output_dir, 'wb') as f:
    np.save(f, vectorised_data)
    np.save(f, un_vectorised_data)

In [ ]:
size = 3000
sample_indices = np.random.randint(0, 69999, size)
sample = large_dataset.data[sample_indices]
ld_positions = 20*np.random.rand(sample.shape[0],2)
stress_ts = vectorised_stress(sample, ld_positions, euclidean)

